# CS4248 Project Group 23

In [1]:
# If you wish to run this on Google Colab, mount the Google drive by running this cell or click the `files` icon on the left navbar
# and click mount Google Drive (it takes some time to load)
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"
# !cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"

In [2]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [3]:
# Unzip raw_data.zip locally
import zipfile
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

Feature Engineering: Capture various features of the text (e.g. punctuation, stopwords, statement length). 
Test out different tokenizers to capture their performance.


In [6]:
import gensim.downloader as api

def load_glove_model():
    glove_model = api.load('glove-wiki-gigaword-300')
    print("Done.",len(glove_model)," words loaded!")
    return glove_model, glove_model.vector_size

glove_model, glove_dim = load_glove_model()

Done. 400000  words loaded!


In [7]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# todo parallelize this in future


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # tokenize
    words = word_tokenize(text)
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Get the GloVe vectors
    vectors = [glove_model[word] for word in words if word in glove_model]
    # If vectors is empty, return a vector of zeros
    if not vectors:
        print("No vectors found for the text: ", text)
        return np.zeros(glove_model.vector_size)
    return np.mean(vectors, axis=0)

In [8]:
train = pd.read_csv("./raw_data/fulltrain.csv", names=['Verdict', 'Text'])
test = pd.read_csv("./raw_data/balancedtest.csv", names=['Verdict', 'Text'])

# Preprocess all documents in the corpus
preprocessed_corpus = [preprocess_text(text) for text in train['Text']]

# Fit the TF-IDF vectorizer on the preprocessed corpus
X_train = np.array([preprocess_text(text) for text in train['Text']]) 

No vectors found for the text:  abfs daksl 
No vectors found for the text:  abfs daksl 


In [9]:
X_test = np.array([preprocess_text(text) for text in test['Text']]) 
X_train = np.abs(X_train)
X_test = np.abs(X_test)

y_train = train['Verdict'].apply(lambda x: 1 if x == 4 else 0) # convert to binary- label 4 = trusted
y_test = test['Verdict'].apply(lambda x: 1 if x == 4 else 0) # convert to binary- label 4 = trusted

Test out different kinds of models and find the most effective architectures.|

In [10]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
model = MultinomialNB()

In [11]:
from sklearn.model_selection import GridSearchCV

# or perform hyperparameter tuning
# Create the hyperparameters grid

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
    'fit_prior': [True, False],  # Whether to learn class prior probabilities or not
    'class_prior': [None, [0.9, 0.1], [0.8, 0.2], [0.7, 0.3], [0.6, 0.4], [0.5, 0.5], [0.4, 0.6], [0.3, 0.7], [0.2, 0.8], [0.1, 0.9]]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

# Train
grid_search.fit(X_train, y_train)


hyperparams = str(grid_search.best_params_)
print (hyperparams)

# Use best model
model = grid_search.best_estimator_

{'alpha': 0.1, 'class_prior': [0.6, 0.4], 'fit_prior': True}


In [12]:
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Training F1 score is:", f1_score(y_train, y_pred_train))
print("Test F1 score is:", f1_score(y_test, y_pred_test))

Training F1 score is: 0.657636302065696
Test F1 score is: 0.6029798422436459


In [13]:
# load a previous model
import joblib
model = joblib.load("./sklearn_models/logisticRegression.pkl")
y_pred = model.predict(x_train)

FileNotFoundError: [Errno 2] No such file or directory: './sklearn_models/logisticRegression.pkl'

In [ ]:
# save the sklearn model
# best model now is {'alpha': 0.1, 'class_prior': [0.6, 0.4], 'fit_prior': True}
import joblib
joblib.dump(model, './sklearn_models/Naive Bayes GloVe '+ str(f1_score(y_test, y_pred_test)) + '.pkl')

['./sklearn_models/Naive Bayes GloVe 0.6029798422436459.pkl']